In [186]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval

In [192]:
df=pd.read_csv("Dataset_ML/Dataset_ML_function.csv")

In [193]:
df.title=df.title.str.lower()## Normalize title to lowercase
df.genres=df.genres.str.replace(" ","")##Removing blank spaces
df.genres=df.genres.apply(literal_eval)## Setting correct datatype in column genres
df["features"]=df.genres

In [194]:
for i in range(df.shape[0]):
    df.genres.iloc[i]=" ".join(df.genres.iloc[i])
for i in range(df.shape[0]):
    df.features.iloc[i]=df.genres.iloc[i]+" "+df.overview.iloc[i]

C:\Users\angeo\AppData\Local\Temp\ipykernel_2444\158336877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.genres.iloc[i]=" ".join(df.genres.iloc[i])
C:\Users\angeo\AppData\Local\Temp\ipykernel_2444\158336877.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.features.iloc[i]=df.genres.iloc[i]+" "+df.overview.iloc[i]


In [195]:
df = df.sort_values("popularity", ascending=False)## Priority by popularity.. due an computational cost if whole dataset is used in ML model
df.drop(["genres","overview","popularity"],inplace=True,axis=1)
df.features=df.features.str.lower()
df.reset_index(drop=True,inplace=True)

In [197]:
df

,title,features
0,minions,family animation adventure comedy minions stua...
1,baby driver,action crime after being coerced into working ...
2,big hero 6,adventure family animation action comedy the s...
3,deadpool,action adventure comedy deadpool tells the ori...
4,guardians of the galaxy vol. 2,action adventure comedy sciencefiction the gua...
...,...,...
39304,terror,"horror dr. vishal, a mad scientist, turns into..."
39305,altar of fire,documentary this film records a 12 day ritual ...
39306,casual relations,drama in rappaportâ€™s dazzling and bizarre fe...
39307,narrien illat,comedy drama music the ups and downs of the po...


In [198]:
df.to_csv("Dataset_API/API7.csv",index=False)

In [200]:
df=pd.read_csv("Dataset_API/API7.csv")

In [201]:
ml=df.head(10000)
ml.reset_index(inplace=True, drop=True)
ml.reset_index(inplace=True)


In [202]:
ml

,index,title,features
0,0,minions,family animation adventure comedy minions stua...
1,1,baby driver,action crime after being coerced into working ...
2,2,big hero 6,adventure family animation action comedy the s...
3,3,deadpool,action adventure comedy deadpool tells the ori...
4,4,guardians of the galaxy vol. 2,action adventure comedy sciencefiction the gua...
...,...,...,...
9995,9995,turn the river,drama a pool shark takes the ultimate gamble w...
9996,9996,wonderful world,comedy drama romance ben singer (matthew brode...
9997,9997,josh,romance drama shah rukh khan and sharad kapoor...
9998,9998,the last command,drama history romance a former imperial russia...


In [203]:
tfidf=TfidfVectorizer(stop_words="english",max_features=10000)
 

In [204]:
tfidf_matrix=tfidf.fit_transform(ml["features"])

In [205]:
tfidf_matrix.shape

(10000, 10000)

In [206]:
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [207]:
ml

,index,title,features
0,0,minions,family animation adventure comedy minions stua...
1,1,baby driver,action crime after being coerced into working ...
2,2,big hero 6,adventure family animation action comedy the s...
3,3,deadpool,action adventure comedy deadpool tells the ori...
4,4,guardians of the galaxy vol. 2,action adventure comedy sciencefiction the gua...
...,...,...,...
9995,9995,turn the river,drama a pool shark takes the ultimate gamble w...
9996,9996,wonderful world,comedy drama romance ben singer (matthew brode...
9997,9997,josh,romance drama shah rukh khan and sharad kapoor...
9998,9998,the last command,drama history romance a former imperial russia...


In [209]:
indices=ml[["title","index"]]

In [210]:
def recomendacion(titulo):
    idx = indices[indices["title"]==titulo]
    if idx.empty== True:
        recommendations=["No data available"]
    else:
        idy = idx["index"].iloc[0]
        sim_scores = list(enumerate(cosine_sim[idy]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
        movie_indices = [i[0] for i in sim_scores]
        recommendations = list(ml['title'].iloc[movie_indices].str.title())
    return {'titulo':titulo,'lista recomendada': recommendations}


In [211]:
recomendacion("toy story")


{'titulo': 'toy story',
 'lista recomendada': ['Toy Story 3',
  'Toy Story 2',
  'The 40 Year Old Virgin',
  'Small Fry',
  'Man On The Moon']}